In [25]:
!pip install fastapi uvicorn transformers torch spacy language-tool-python nltk scikit-learn praat-parselmouth textstat sentence-transformers pyngrok pydantic nest_asyncio

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-14' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 579, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    s

In [ ]:
!pip install fastapi uvicorn pyngrok pydantic

In [ ]:
!pip install nest_asyncio

In [26]:
import re
from sentence_transformers import SentenceTransformer, util
import language_tool_python
from datetime import datetime

In [27]:
import spacy
import language_tool_python
import textstat
import nltk
from transformers import pipeline
from sklearn.feature_extraction.text import TfidfVectorizer


# Load mô hình spaCy để nhận diện Candidate & Examiner
nlp = spacy.load("en_core_web_sm")

# Load BERT để phân loại câu trả lời
classifier = pipeline("text-classification", model="textattack/bert-base-uncased-SST-2")

# Load công cụ kiểm tra ngữ pháp
grammar_tool = language_tool_python.LanguageTool('en-UK')


/usr/local/lib/python3.11/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)
Device set to use cpu


hàm chuẩn hóa dữ liệu

In [28]:
def parse_transcription(transcription):
    # Tách các phần dựa theo mẫu '--- Part X ---'
    parts = re.split(r'--- Part (\d+) ---', transcription)
    result = []  # List chứa các list con, mỗi list con có dạng: [part, question, response]

    # Bắt đầu từ index 1, duyệt theo cặp: số part và nội dung của part
    for i in range(1, len(parts), 2):
        try:
            part_number = int(parts[i].strip())
        except ValueError:
            continue
        part_text = parts[i + 1].strip()

        # Tìm kiếm các cặp câu hỏi - câu trả lời
        pattern = re.compile(r'examiner: (.*?)\n(candidate: .*?)(?=\nexaminer:|\n--- Part|\Z)', re.DOTALL)
        pairs = pattern.findall(part_text)

        for question, response in pairs:
            # Loại bỏ tiền tố "candidate: " trong câu trả lời
            response_clean = response.replace("candidate: ", "").strip()
            # Append kết quả dưới dạng list con: [part, question, response]
            result.append([part_number, question.strip(), response_clean])

    return result

hàm xem độ tương quan của câu trả lời

In [29]:
def analyze_answer_relevance(data_list):
    try:
        # Initialize the model
        model = SentenceTransformer('all-MiniLM-L6-v2')

        # Print header
        print(f"Current Date and Time (UTC - YYYY-MM-DD HH:MM:SS formatted): {datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"Current User's Login: NgPcAnhh")
        print("=" * 100)

        def evaluate_relevance(question, answer):
            """Calculate cosine similarity between question and answer embeddings."""
            q_embedding = model.encode(question, convert_to_tensor=True)
            a_embedding = model.encode(answer, convert_to_tensor=True)

            similarity = util.cos_sim(q_embedding, a_embedding).item()
            similarity_percentage = similarity * 100

            if similarity < 0.2:
                label = "Not Relevant"
            elif similarity < 0.5:
                label = "Neutral"
            else:
                label = "Relevant"

            return similarity_percentage, label

        # Analyze each entry
        for entry in data_list:
            part_number = entry[0]
            question = entry[1]
            answer = entry[2]

            # Calculate relevance
            relevance_score, relevance_label = evaluate_relevance(question, answer)

            # Print analysis for this entry
            print(f"\nPart {part_number}")
            print(f"Question: {question}")
            print(f"Answer: {answer}")
            print(f"Relevance Score: {relevance_score:.2f}%")
            print(f"Label: {relevance_label}")
            print("-" * 100)

    except Exception as e:
        print(f"Error occurred during analysis: {str(e)}")

hàm check lỗi ngữ pháp

In [30]:
def analyze_all_answers(data_list):
    try:
        # Initialize the grammar tool
        grammar_tool = language_tool_python.LanguageTool('en-UK')

        print(f"\n====== Grammar Analysis Report ======")
        print(f"Analysis Time: {datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"Total entries to analyze: {len(data_list)}")
        print("=" * 40)

        # Dictionary to store all errors
        all_results = {}

        # Analyze each entry in the data list
        for entry in data_list:
            part_number = entry[0]
            question = entry[1]
            answer_text = entry[2]  # Last element is always the answer

            print(f"\n\n--- Part {part_number} ---")
            print(f"Question: {question}")
            print(f"\nAnswer: {answer_text}")
            print("\nGrammar Analysis:")

            # Get all matches (errors)
            matches = grammar_tool.check(answer_text)

            # Filter out punctuation errors
            excluded_rules = [
                "COMMA_COMPOUND_SENTENCE",
                "COMMA_PARENTHESIS_WHITESPACE",
                "PUNCTUATION_COMMA",
                "PUNCTUATION_PERIOD",
                "PUNCTUATION_QUOTATION",
                "PUNCTUATION_WHITESPACE",
                "COMMA_PARENTHESIS_START_WHITESPACE",
                "COLON_WHITESPACE",
                "EXTRA_COMMA",
                "MISSING_COMMA",
                "SEMICOLON_WHITESPACE",
                "EXTRA_PERIOD",
                "MISSING_PERIOD",
                "EXTRA_WHITESPACE",
                "MISSING_WHITESPACE",
                "WHITESPACE_RULE"
            ]

            # Create markers for error positions
            error_markers = list(" " * len(answer_text))
            errors_found = []

            for match in matches:
                if not match.ruleId.startswith("PUNCT") and match.ruleId not in excluded_rules:
                    # Mark error position
                    for pos in range(match.offset, match.offset + match.errorLength):
                        if pos < len(error_markers):
                            error_markers[pos] = "^"

                    error_info = {
                        "error_type": match.ruleId,
                        "message": match.message,
                        "context": match.context,
                        "suggestion": match.replacements[0] if match.replacements else "No suggestion available",
                        "position": (match.offset, match.offset + match.errorLength),
                        "text": answer_text[match.offset:match.offset + match.errorLength]
                    }
                    errors_found.append(error_info)

            # Store results for this part
            all_results[part_number] = {
                "question": question,
                "answer": answer_text,
                "errors": errors_found
            }

            # Print error details
            if errors_found:
                print(f"\nFound {len(errors_found)} errors:")
                for i, error in enumerate(errors_found, 1):
                    print(f"\n{i}. Error Type: {error['error_type']}")
                    print(f"   Found Text: \"{error['text']}\"")
                    print(f"   Message: {error['message']}")
                    print(f"   Suggestion: {error['suggestion']}")

                # Print text with error markers
                print("\nError Locations:")
                print(answer_text)
                print("".join(error_markers))
            else:
                print("No grammar errors found in this answer!")

            print("-" * 40)

        # Print overall summary
        print("\n====== Overall Summary ======")
        total_errors = sum(len(part_data["errors"]) for part_data in all_results.values())
        print(f"Total parts analyzed: {len(data_list)}")
        print(f"Total errors found: {total_errors}")
        print(f"Average errors per part: {total_errors/len(data_list):.2f}")

        return all_results

    except Exception as e:
        print(f"Error occurred during grammar analysis: {str(e)}")
        return {}



API

In [35]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import uvicorn
from pyngrok import ngrok
import nest_asyncio
import re
from sentence_transformers import SentenceTransformer, util
from datetime import datetime
import language_tool_python

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

app = FastAPI()

# Mô tả dữ liệu đầu vào
class TranscriptionInput(BaseModel):
    transcription: list

# Your provided function, unchanged
def parse_transcription(transcription):
    # Tách các phần dựa theo mẫu '--- Part X ---'
    parts = re.split(r'--- Part (\d+) ---', transcription)
    result = []  # List chứa các list con, mỗi list con có dạng: [part, question, response]

    # Bắt đầu từ index 1, duyệt theo cặp: số part và nội dung của part
    for i in range(1, len(parts), 2):
        try:
            part_number = int(parts[i].strip())
        except ValueError:
            continue
        part_text = parts[i + 1].strip()

        # Tìm kiếm các cặp câu hỏi - câu trả lời
        pattern = re.compile(r'examiner: (.*?)\n(candidate: .*?)(?=\nexaminer:|\n--- Part|\Z)', re.DOTALL)
        pairs = pattern.findall(part_text)

        for question, response in pairs:
            # Loại bỏ tiền tố "candidate: " trong câu trả lời
            response_clean = response.replace("candidate: ", "").strip()
            # Append kết quả dưới dạng list con: [part, question, response]
            result.append([part_number, question.strip(), response_clean])

    return result

# Your provided function, modified to return the data instead of printing
def analyze_answer_relevance(data_list):
    try:
        # Initialize the model
        model = SentenceTransformer('all-MiniLM-L6-v2')

        # Result list
        results = []

        def evaluate_relevance(question, answer):
            """Calculate cosine similarity between question and answer embeddings."""
            q_embedding = model.encode(question, convert_to_tensor=True)
            a_embedding = model.encode(answer, convert_to_tensor=True)

            similarity = util.cos_sim(q_embedding, a_embedding).item()
            similarity_percentage = similarity * 100

            if similarity < 0.2:
                label = "Not Relevant"
            elif similarity < 0.5:
                label = "Neutral"
            else:
                label = "Relevant"

            return similarity_percentage, label

        # Analyze each entry
        for entry in data_list:
            part_number = entry[0]
            question = entry[1]
            answer = entry[2]

            # Calculate relevance
            relevance_score, relevance_label = evaluate_relevance(question, answer)

            # Store analysis for this entry
            results.append({
                "part_number": part_number,
                "question": question,
                "answer": answer,
                "relevance_score": relevance_score,
                "label": relevance_label
            })

        return results

    except Exception as e:
        return {"error": str(e)}

# Your provided function, modified to return the data instead of printing
def analyze_all_answers(data_list):
    try:
        # Initialize the grammar tool
        grammar_tool = language_tool_python.LanguageTool('en-UK')

        # Dictionary to store all errors
        all_results = {}

        # Analyze each entry in the data list
        for entry in data_list:
            part_number = entry[0]
            question = entry[1]
            answer_text = entry[2]  # Last element is always the answer

            # Get all matches (errors)
            matches = grammar_tool.check(answer_text)

            # Filter out punctuation errors
            excluded_rules = [
                "COMMA_COMPOUND_SENTENCE",
                "COMMA_PARENTHESIS_WHITESPACE",
                "PUNCTUATION_COMMA",
                "PUNCTUATION_PERIOD",
                "PUNCTUATION_QUOTATION",
                "PUNCTUATION_WHITESPACE",
                "COMMA_PARENTHESIS_START_WHITESPACE",
                "COLON_WHITESPACE",
                "EXTRA_COMMA",
                "MISSING_COMMA",
                "SEMICOLON_WHITESPACE",
                "EXTRA_PERIOD",
                "MISSING_PERIOD",
                "EXTRA_WHITESPACE",
                "MISSING_WHITESPACE",
                "WHITESPACE_RULE"
            ]

            # Create markers for error positions
            error_markers = list(" " * len(answer_text))
            errors_found = []

            for match in matches:
                if not match.ruleId.startswith("PUNCT") and match.ruleId not in excluded_rules:
                    # Mark error position
                    for pos in range(match.offset, match.offset + match.errorLength):
                        if pos < len(error_markers):
                            error_markers[pos] = "^"

                    error_info = {
                        "error_type": match.ruleId,
                        "message": match.message,
                        "context": match.context,
                        "suggestion": match.replacements[0] if match.replacements else "No suggestion available",
                        "position": (match.offset, match.offset + match.errorLength),
                        "text": answer_text[match.offset:match.offset + match.errorLength]
                    }
                    errors_found.append(error_info)

            # Store results for this part
            all_results[part_number] = {
                "question": question,
                "answer": answer_text,
                "errors": errors_found
            }

        return all_results

    except Exception as e:
        return {"error": str(e)}

@app.post("/process_transcription/")
def process_transcription(data: TranscriptionInput):
    transcription = data.transcription
    transcription_str = '\n'.join(transcription)
    parsed_transcription = parse_transcription(transcription_str)

    relevance_result = analyze_answer_relevance(parsed_transcription)
    analysis_result = analyze_all_answers(parsed_transcription)

    return {"relevance": relevance_result, "analysis": analysis_result}

@app.get("/public_url/")
def get_public_url():
    # Set your ngrok API token here
    ngrok.set_auth_token("your_ngrok_api_token")

    # Create a tunnel for your application
    url = ngrok.connect(8000).public_url

    return {"public_url": url}

# Public API bằng ngrok
if __name__ == "__main__":
    # Set your ngrok API token here
    ngrok.set_auth_token("2sWjZTSTemDeBnksDgYJvraqVku_7aYAKKMyNoNhKzoUxhHFU")

    # Create a tunnel for your application
    url = ngrok.connect(8000).public_url
    print(f"Public URL: {url}")

    # Run the FastAPI application
    uvicorn.run(app, host="0.0.0.0", port=8000)

Public URL: https://4621-34-73-38-44.ngrok-free.app


INFO:     Started server process [1047]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-38' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 579, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
   

INFO:     42.116.198.240:0 - "POST /process_transcription/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1047]
